# Algorithmic Strategies Mini-Project

**Lab Assignment 2 — ENCA351**

This notebook implements four real-world problems using different algorithmic strategies:

1. Scheduling TV Commercials (Greedy - Job Sequencing)
2. 0/1 Knapsack (Dynamic Programming)
3. Sudoku Solver (Backtracking)
4. Password Cracking (Brute-Force)

*Profiling mode:* Execution time only (using `time.perf_counter`).

All code cells are self-contained and can be executed sequentially.

In [ ]:
# Standard imports and timing utility
import time
import random
import itertools
import matplotlib.pyplot as plt
import numpy as np
import statistics

def time_function(func, *args, repeats=3, **kwargs):
    times = []
    for _ in range(repeats):
        t0 = time.perf_counter()
        res = func(*args, **kwargs)
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return statistics.mean(times), res if 'res' in locals() else None


## Problem 1 — Scheduling TV Commercials (Greedy: Job Sequencing)

**Goal:** Given ads with (id, deadline, profit), schedule non-overlapping ads to maximize revenue.

Approach: Greedy — sort by profit descending and place each job in the latest available slot before its deadline.

In [ ]:
def job_sequencing(ads):
    """ads: list of tuples (id, deadline, profit)"""
    # sort by profit descending
    ads_sorted = sorted(ads, key=lambda x: x[2], reverse=True)
    max_deadline = max(ad[1] for ad in ads_sorted)
    slots = [None] * (max_deadline + 1)  # 1-based slots
    total_profit = 0
    selected = []
    for ad in ads_sorted:
        ad_id, deadline, profit = ad
        # find a slot for this ad (latest possible before deadline)
        for d in range(min(max_deadline, deadline), 0, -1):
            if slots[d] is None:
                slots[d] = ad_id
                total_profit += profit
                selected.append(ad)
                break
    return selected, total_profit, slots

# Example usage
sample_ads = [('A',2,100), ('B',1,19), ('C',2,27), ('D',1,25), ('E',3,15)]
sel, profit, slots = job_sequencing(sample_ads)
print('Selected ads:', sel)
print('Total profit:', profit)
print('Slots:', slots)

In [ ]:
# Visualization: number of ads vs revenue (simulate varying number of ads)
def simulate_job_sequencing(num_ads_list):
    results = []
    for n in num_ads_list:
        ads = []
        for i in range(n):
            ad_id = f'ad{i}'
            deadline = random.randint(1, max(1, n//5))  # deadlines smaller than n
            profit = random.randint(1, 500)
            ads.append((ad_id, deadline, profit))
        t, (selected, total_profit, slots) = time_function(job_sequencing, ads)
        results.append((n, total_profit))
    return results

num_ads_list = [10, 20, 50, 100, 200]
results_js = simulate_job_sequencing(num_ads_list)

# plot
ns = [r[0] for r in results_js]
profits = [r[1] for r in results_js]
plt.figure(figsize=(8,4))
plt.plot(ns, profits, marker='o')
plt.xlabel('Number of ads')
plt.ylabel('Total revenue (simulated)')
plt.title('Job Sequencing: Number of ads vs Revenue')
plt.grid(True)
plt.show()

## Problem 2 — Maximizing Profit with Limited Budget (0/1 Knapsack DP)

**Goal:** Choose items with given costs (weights) and values (profits) under a budget (capacity) to maximize profit.

Approach: Bottom-up dynamic programming (classic 0/1 Knapsack).

In [ ]:
def knapsack_01(weights, values, capacity):
    n = len(weights)
    dp = [[0]*(capacity+1) for _ in range(n+1)]
    for i in range(1, n+1):
        wt = weights[i-1]
        val = values[i-1]
        for w in range(capacity+1):
            dp[i][w] = dp[i-1][w]
            if wt <= w:
                dp[i][w] = max(dp[i][w], dp[i-1][w-wt] + val)
    # reconstruct chosen items
    res_val = dp[n][capacity]
    w = capacity
    chosen = []
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i-1][w]:
            chosen.append(i-1)
            w -= weights[i-1]
    chosen.reverse()
    return res_val, chosen

# Example usage
weights = [2,3,4,5]
values = [3,4,5,8]
capacity = 5
max_profit, chosen = knapsack_01(weights, values, capacity)
print('Max profit:', max_profit)
print('Chosen item indices:', chosen)

In [ ]:
# Visualization: profit vs capacity (simulate)
def simulate_knapsack(num_items, capacities):
    # random items
    weights = [random.randint(1, 20) for _ in range(num_items)]
    values = [random.randint(1, 100) for _ in range(num_items)]
    results = []
    for cap in capacities:
        t, (res_val, chosen) = time_function(knapsack_01, weights, values, cap)
        results.append((cap, res_val))
    return weights, values, results

capacities = [10, 20, 30, 40, 50]
weights, values, results_knap = simulate_knapsack(15, capacities)
caps = [r[0] for r in results_knap]
profits = [r[1] for r in results_knap]

plt.figure(figsize=(8,4))
plt.plot(caps, profits, marker='o')
plt.xlabel('Capacity (Budget)')
plt.ylabel('Maximum profit (simulated)')
plt.title('0/1 Knapsack: Capacity vs Max Profit')
plt.grid(True)
plt.show()

## Problem 3 — Solving Sudoku (Backtracking)

**Goal:** Complete a partially filled 9x9 Sudoku grid using recursive backtracking with constraint checks.

In [ ]:
# Sudoku solver
def find_empty(grid):
    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0:
                return i, j
    return None

def valid(grid, row, col, num):
    # check row and column
    if any(grid[row][c] == num for c in range(9)): return False
    if any(grid[r][col] == num for r in range(9)): return False
    # check 3x3 box
    br = (row//3)*3
    bc = (col//3)*3
    for r in range(br, br+3):
        for c in range(bc, bc+3):
            if grid[r][c] == num:
                return False
    return True

def sudoku_solve(grid):
    empty = find_empty(grid)
    if not empty:
        return True
    row, col = empty
    for num in range(1,10):
        if valid(grid, row, col, num):
            grid[row][col] = num
            if sudoku_solve(grid):
                return True
            grid[row][col] = 0
    return False

# Sample Sudoku (0 denotes empty)
sample_grid = [
    [5,3,0,0,7,0,0,0,0],
    [6,0,0,1,9,5,0,0,0],
    [0,9,8,0,0,0,0,6,0],
    [8,0,0,0,6,0,0,0,3],
    [4,0,0,8,0,3,0,0,1],
    [7,0,0,0,2,0,0,0,6],
    [0,6,0,0,0,0,2,8,0],
    [0,0,0,4,1,9,0,0,5],
    [0,0,0,0,8,0,0,7,9]
]

import copy
grid_copy = copy.deepcopy(sample_grid)
t0 = time.perf_counter()
solved = sudoku_solve(grid_copy)
t1 = time.perf_counter()
print('Solved:', solved, 'Time:', t1 - t0)
if solved:
    for row in grid_copy:
        print(row)

In [ ]:
# Optional: time vs number of empties (simulate by removing entries)
def simulate_sudoku(num_blanks_list):
    results = []
    base = [row[:] for row in sample_grid]
    for blanks in num_blanks_list:
        # create a copy and blank out 'blanks' additional cells randomly
        g = [row[:] for row in base]
        # flatten positions
        positions = [(i,j) for i in range(9) for j in range(9) if g[i][j] != 0]
        random.shuffle(positions)
        for k in range(min(blanks, len(positions))):
            i,j = positions[k]
            g[i][j] = 0
        t0 = time.perf_counter()
        solved = sudoku_solve(g)
        t1 = time.perf_counter()
        results.append((blanks, t1 - t0, solved))
    return results

num_blanks_list = [5, 10, 15, 20, 30]
results_sudoku = simulate_sudoku(num_blanks_list)
results_sudoku

## Problem 4 — Password Cracking (Brute-Force)

**Goal:** Try all possible combinations from a given charset to find the target password. This is intentionally naive and demonstrates exponential growth with password length.

In [ ]:
def brute_force_crack(target, charset, max_len=6):
    attempts = 0
    for length in range(1, max_len+1):
        for combo in itertools.product(charset, repeat=length):
            attempts += 1
            candidate = ''.join(combo)
            if candidate == target:
                return candidate, attempts
    return None, attempts

# Demo with small charset/length
charset = 'ab12'
target = '1b'
t0 = time.perf_counter()
found, attempts = brute_force_crack(target, charset, max_len=4)
t1 = time.perf_counter()
print('Found:', found, 'Attempts:', attempts, 'Time:', t1-t0)

In [ ]:
# Visualization: time vs password length (simulated small charset)
def simulate_bruteforce(target_len_list, charset='abc1'):
    results = []
    for L in target_len_list:
        # create a random target of length L
        target = ''.join(random.choice(charset) for _ in range(L))
        t0 = time.perf_counter()
        found, attempts = brute_force_crack(target, charset, max_len=L)
        t1 = time.perf_counter()
        results.append((L, t1 - t0, attempts))
    return results

target_len_list = [1,2,3,4]
results_bf = simulate_bruteforce(target_len_list, charset='ab1')
lengths = [r[0] for r in results_bf]
times = [r[1] for r in results_bf]

plt.figure(figsize=(8,4))
plt.plot(lengths, times, marker='o')
plt.xlabel('Password length')
plt.ylabel('Time (s)')
plt.title('Brute-Force: Time vs Password Length (small charset)')
plt.grid(True)
plt.show()

results_bf

## Summary & Next Steps

- The notebook demonstrates four algorithmic strategies applied to practical problems.
- Greedy and Dynamic Programming show good trade-offs for scheduling and budget problems respectively.
- Backtracking is powerful for constraint-satisfaction problems like Sudoku but may be slow for hard instances.
- Brute-force password cracking grows exponentially with password length and charset size.

**Next steps for submission:**
- Save this notebook as `algo_strategies_notebook.ipynb` and push to your GitHub repository alongside README.md and requirements.txt.
